<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Lab: Custom Training Loops in Keras**


Estimated time needed: **30** minutes


In this lab, you will learn to implement a basic custom training loop in Keras. 


## Objectives

By the end of this lab, you will: 

- Set up the environment 

- Define the neural network model 

- Define the Loss Function and Optimizer 

- Implement the custom training loop 

- Enhance the custom training loop by adding an accuracy metric to monitor model performance 

- Implement a custom callback to log additional metrics and information during training


----


## Step-by-Step Instructions:


### Exercise 1: Basic custom training loop: 

#### 1. Set Up the Environment:

- Import necessary libraries. 

- Load and preprocess the MNIST dataset. 


In [1]:
!pip install tensorflow numpy

In [4]:
import os
import warnings
import tensorflow as tf 
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.callbacks import Callback
import numpy as np

# Suppress all Python warnings
warnings.filterwarnings('ignore')

# Set TensorFlow log level to suppress warnings and info messages
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Step 1: Set Up the Environment
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data() 
x_train, x_test = x_train / 255.0, x_test / 255.0 
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)
train_dataset


<_BatchDataset element_spec=(TensorSpec(shape=(None, 28, 28), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.uint8, name=None))>

#### 2. Define the model: 

Create a simple neural network model with a Flatten layer followed by two Dense layers. 


In [5]:
# Step 2: Define the Model

model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10)
])


#### 3. Define Loss Function and Optimizer: 

- Use Sparse Categorical Crossentropy for the loss function. 
- Use the Adam optimizer. 


In [6]:
# Step 3: Define Loss Function and Optimizer

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) 
optimizer = tf.keras.optimizers.Adam()


#### 4. Implement the Custom Training Loop: 

- Iterate over the dataset for a specified number of epochs. 
- Compute the loss and apply gradients to update the model's weights. 


In [7]:
# Step 4: Implement the Custom Training Loop

epochs = 2
# train_dataset = train_dataset.repeat(epochs)
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)
for epoch in range(epochs):
    print(f'Start of epoch {epoch + 1}')

    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch_train, training=True)  # Forward pass
            loss_value = loss_fn(y_batch_train, logits)  # Compute loss

        # Compute gradients and update weights
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Logging the loss every 200 steps
        if step % 200 == 0:
            print(f'Epoch {epoch + 1} Step {step}: Loss = {loss_value.numpy()}')


Start of epoch 1
Epoch 1 Step 0: Loss = 2.392188787460327
Epoch 1 Step 200: Loss = 0.3698611557483673
Epoch 1 Step 400: Loss = 0.19799195230007172
Epoch 1 Step 600: Loss = 0.1945883333683014
Epoch 1 Step 800: Loss = 0.1792069971561432
Epoch 1 Step 1000: Loss = 0.4817534387111664
Epoch 1 Step 1200: Loss = 0.18179628252983093
Epoch 1 Step 1400: Loss = 0.23798878490924835
Epoch 1 Step 1600: Loss = 0.19588863849639893
Epoch 1 Step 1800: Loss = 0.1275886744260788


2024-12-14 19:23:13.943457: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Start of epoch 2
Epoch 2 Step 0: Loss = 0.10252457857131958
Epoch 2 Step 200: Loss = 0.20024023950099945
Epoch 2 Step 400: Loss = 0.12598346173763275
Epoch 2 Step 600: Loss = 0.0592510849237442
Epoch 2 Step 800: Loss = 0.10988759994506836
Epoch 2 Step 1000: Loss = 0.29947590827941895
Epoch 2 Step 1200: Loss = 0.12959270179271698
Epoch 2 Step 1400: Loss = 0.13667641580104828
Epoch 2 Step 1600: Loss = 0.13044901192188263
Epoch 2 Step 1800: Loss = 0.04647912085056305


2024-12-14 19:24:09.782381: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


### Exercise 2: Adding Accuracy Metric:

Enhance the custom training loop by adding an accuracy metric to monitor model performance. 

#### 1. Set Up the Environment: 

Follow the setup from Exercise 1. 


In [8]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Flatten 

# Step 1: Set Up the Environment
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize the pixel values to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0 

# Create a batched dataset for training
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)


#### 2. Define the Model: 
Use the same model as in Exercise 1. 


In [9]:
# Step 2: Define the Model

model = Sequential([ 
    Flatten(input_shape=(28, 28)),  # Flatten the input to a 1D vector
    Dense(128, activation='relu'),  # First hidden layer with 128 neurons and ReLU activation
    Dense(10)  # Output layer with 10 neurons for the 10 classes (digits 0-9)
])


#### 3. Define the loss function, optimizer, and metric: 

- Use Sparse Categorical Crossentropy for the loss function and Adam optimizer. 

- Add Sparse Categorical Accuracy as a metric. 


In [10]:
# Step 3: Define Loss Function, Optimizer, and Metric

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)  # Loss function for multi-class classification
optimizer = tf.keras.optimizers.Adam()  # Adam optimizer for efficient training
accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy()  # Metric to track accuracy during training


#### 4. Implement the custom training loop with accuracy: 

Track the accuracy during training and print it at regular intervals. 


In [11]:
# Step 4: Implement the Custom Training Loop with Accuracy

epochs = 5  # Number of epochs for training

for epoch in range(epochs):
    print(f'Start of epoch {epoch + 1}')
    
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            # Forward pass: Compute predictions
            logits = model(x_batch_train, training=True)
            # Compute loss
            loss_value = loss_fn(y_batch_train, logits)
        
        # Compute gradients
        grads = tape.gradient(loss_value, model.trainable_weights)
        # Apply gradients to update model weights
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        
        # Update the accuracy metric
        accuracy_metric.update_state(y_batch_train, logits)

        # Log the loss and accuracy every 200 steps
        if step % 200 == 0:
            print(f'Epoch {epoch + 1} Step {step}: Loss = {loss_value.numpy()} Accuracy = {accuracy_metric.result().numpy()}')
    
    # Reset the metric at the end of each epoch
    accuracy_metric.reset_state()


Start of epoch 1
Epoch 1 Step 0: Loss = 2.368044137954712 Accuracy = 0.125
Epoch 1 Step 200: Loss = 0.40991589426994324 Accuracy = 0.8358209133148193
Epoch 1 Step 400: Loss = 0.19177937507629395 Accuracy = 0.8675187230110168
Epoch 1 Step 600: Loss = 0.2177790403366089 Accuracy = 0.8832674622535706
Epoch 1 Step 800: Loss = 0.168302983045578 Accuracy = 0.8960674405097961
Epoch 1 Step 1000: Loss = 0.4399988055229187 Accuracy = 0.9032217860221863
Epoch 1 Step 1200: Loss = 0.1875588297843933 Accuracy = 0.9092422723770142
Epoch 1 Step 1400: Loss = 0.24257495999336243 Accuracy = 0.9137669801712036
Epoch 1 Step 1600: Loss = 0.19159090518951416 Accuracy = 0.9169464111328125
Epoch 1 Step 1800: Loss = 0.1333625167608261 Accuracy = 0.920911967754364


2024-12-14 19:27:39.265119: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Start of epoch 2
Epoch 2 Step 0: Loss = 0.08124369382858276 Accuracy = 0.96875
Epoch 2 Step 200: Loss = 0.17925435304641724 Accuracy = 0.9589552283287048
Epoch 2 Step 400: Loss = 0.09158799052238464 Accuracy = 0.9579176902770996
Epoch 2 Step 600: Loss = 0.05641049146652222 Accuracy = 0.9598585963249207
Epoch 2 Step 800: Loss = 0.1354137808084488 Accuracy = 0.9606741666793823
Epoch 2 Step 1000: Loss = 0.2869679927825928 Accuracy = 0.9611638188362122
Epoch 2 Step 1200: Loss = 0.09014157950878143 Accuracy = 0.9619848132133484
Epoch 2 Step 1400: Loss = 0.11740605533123016 Accuracy = 0.9628167152404785
Epoch 2 Step 1600: Loss = 0.1411960870027542 Accuracy = 0.9627576470375061
Epoch 2 Step 1800: Loss = 0.0910390242934227 Accuracy = 0.9634925127029419


2024-12-14 19:28:38.231902: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Start of epoch 3
Epoch 3 Step 0: Loss = 0.052728913724422455 Accuracy = 1.0
Epoch 3 Step 200: Loss = 0.11610399931669235 Accuracy = 0.9738805890083313
Epoch 3 Step 400: Loss = 0.08634345978498459 Accuracy = 0.972490668296814
Epoch 3 Step 600: Loss = 0.050940174609422684 Accuracy = 0.9729617238044739
Epoch 3 Step 800: Loss = 0.07757169008255005 Accuracy = 0.973665714263916
Epoch 3 Step 1000: Loss = 0.15987473726272583 Accuracy = 0.9737762212753296
Epoch 3 Step 1200: Loss = 0.07096119970083237 Accuracy = 0.9743703007698059
Epoch 3 Step 1400: Loss = 0.07092946767807007 Accuracy = 0.974504828453064
Epoch 3 Step 1600: Loss = 0.10036203265190125 Accuracy = 0.9740787148475647
Epoch 3 Step 1800: Loss = 0.04941511154174805 Accuracy = 0.9747015833854675


2024-12-14 19:29:36.854471: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Start of epoch 4
Epoch 4 Step 0: Loss = 0.03710586577653885 Accuracy = 1.0
Epoch 4 Step 200: Loss = 0.09036020934581757 Accuracy = 0.9822761416435242
Epoch 4 Step 400: Loss = 0.06713037937879562 Accuracy = 0.9809850454330444
Epoch 4 Step 600: Loss = 0.04407867416739464 Accuracy = 0.9818011522293091
Epoch 4 Step 800: Loss = 0.0352669395506382 Accuracy = 0.9819756746292114
Epoch 4 Step 1000: Loss = 0.1435191035270691 Accuracy = 0.982017993927002
Epoch 4 Step 1200: Loss = 0.05643458291888237 Accuracy = 0.9819421172142029
Epoch 4 Step 1400: Loss = 0.028263434767723083 Accuracy = 0.9818210005760193
Epoch 4 Step 1600: Loss = 0.05991660803556442 Accuracy = 0.9815154671669006
Epoch 4 Step 1800: Loss = 0.02603719010949135 Accuracy = 0.9819371104240417


2024-12-14 19:30:36.396512: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Start of epoch 5
Epoch 5 Step 0: Loss = 0.02912682294845581 Accuracy = 1.0
Epoch 5 Step 200: Loss = 0.05931224673986435 Accuracy = 0.986940324306488
Epoch 5 Step 400: Loss = 0.05214372277259827 Accuracy = 0.986440122127533
Epoch 5 Step 600: Loss = 0.029306095093488693 Accuracy = 0.987052857875824
Epoch 5 Step 800: Loss = 0.01705067791044712 Accuracy = 0.9871254563331604
Epoch 5 Step 1000: Loss = 0.08604712784290314 Accuracy = 0.9874188303947449
Epoch 5 Step 1200: Loss = 0.05085700750350952 Accuracy = 0.9872762560844421
Epoch 5 Step 1400: Loss = 0.014428183436393738 Accuracy = 0.9870851039886475
Epoch 5 Step 1600: Loss = 0.03378843143582344 Accuracy = 0.9868051409721375
Epoch 5 Step 1800: Loss = 0.017747074365615845 Accuracy = 0.9869690537452698


2024-12-14 19:31:57.680783: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


### Exercise 3: Custom Callback for Advanced Logging: 

Implement a custom callback to log additional metrics and information during training. 

#### 1. Set Up the Environment: 

Follow the setup from Exercise 1.


In [12]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Flatten 

# Step 1: Set Up the Environment
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize the pixel values to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0 

# Create a batched dataset for training
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)


#### 2. Define the Model: 

Use the same model as in Exercise 1. 


In [13]:
# Step 2: Define the Model

model = Sequential([
    Flatten(input_shape=(28, 28)),  # Flatten the input to a 1D vector
    Dense(128, activation='relu'),  # First hidden layer with 128 neurons and ReLU activation
    Dense(10)  # Output layer with 10 neurons for the 10 classes (digits 0-9)
])


#### 3. Define Loss Function, Optimizer, and Metric: 

- Use Sparse Categorical Crossentropy for the loss function and Adam optimizer. 

- Add Sparse Categorical Accuracy as a metric. 


In [14]:
# Step 3: Define Loss Function, Optimizer, and Metric

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)  # Loss function for multi-class classification
optimizer = tf.keras.optimizers.Adam()  # Adam optimizer for efficient training
accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy()  # Metric to track accuracy during training


#### 4. Implement the custom training loop with custom callback: 

Create a custom callback to log additional metrics at the end of each epoch.


In [15]:
from tensorflow.keras.callbacks import Callback

# Step 4: Implement the Custom Callback 
class CustomCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        print(f'End of epoch {epoch + 1}, loss: {logs.get("loss")}, accuracy: {logs.get("accuracy")}')


In [16]:
# Step 5: Implement the Custom Training Loop with Custom Callback

epochs = 2
custom_callback = CustomCallback()  # Initialize the custom callback

for epoch in range(epochs):
    print(f'Start of epoch {epoch + 1}')
    
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            # Forward pass: Compute predictions
            logits = model(x_batch_train, training=True)
            # Compute loss
            loss_value = loss_fn(y_batch_train, logits)
        
        # Compute gradients
        grads = tape.gradient(loss_value, model.trainable_weights)
        # Apply gradients to update model weights
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        
        # Update the accuracy metric
        accuracy_metric.update_state(y_batch_train, logits)

        # Log the loss and accuracy every 200 steps
        if step % 200 == 0:
            print(f'Epoch {epoch + 1} Step {step}: Loss = {loss_value.numpy()} Accuracy = {accuracy_metric.result().numpy()}')
    
    # Call the custom callback at the end of each epoch
    custom_callback.on_epoch_end(epoch, logs={'loss': loss_value.numpy(), 'accuracy': accuracy_metric.result().numpy()})
    
    # Reset the metric at the end of each epoch
    accuracy_metric.reset_state()  # Use reset_state() instead of reset_states()


Start of epoch 1
Epoch 1 Step 0: Loss = 2.3273227214813232 Accuracy = 0.09375
Epoch 1 Step 200: Loss = 0.40633946657180786 Accuracy = 0.8393967747688293
Epoch 1 Step 400: Loss = 0.20144127309322357 Accuracy = 0.8702462315559387
Epoch 1 Step 600: Loss = 0.18538732826709747 Accuracy = 0.8853473663330078
Epoch 1 Step 800: Loss = 0.15534910559654236 Accuracy = 0.8970037698745728
Epoch 1 Step 1000: Loss = 0.46044981479644775 Accuracy = 0.9041271209716797
Epoch 1 Step 1200: Loss = 0.1412172019481659 Accuracy = 0.9102310538291931
Epoch 1 Step 1400: Loss = 0.2495177537202835 Accuracy = 0.9149045348167419
Epoch 1 Step 1600: Loss = 0.2377534806728363 Accuracy = 0.9179028868675232
Epoch 1 Step 1800: Loss = 0.2057649791240692 Accuracy = 0.9219704270362854


2024-12-14 19:32:59.167149: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


End of epoch 1, loss: 0.03805457800626755, accuracy: 0.9239000082015991
Start of epoch 2
Epoch 2 Step 0: Loss = 0.07352970540523529 Accuracy = 1.0
Epoch 2 Step 200: Loss = 0.2065155953168869 Accuracy = 0.9619091749191284
Epoch 2 Step 400: Loss = 0.12999442219734192 Accuracy = 0.9590866565704346
Epoch 2 Step 600: Loss = 0.058244429528713226 Accuracy = 0.9608985185623169
Epoch 2 Step 800: Loss = 0.08151639997959137 Accuracy = 0.9618836045265198
Epoch 2 Step 1000: Loss = 0.29307907819747925 Accuracy = 0.9623501300811768
Epoch 2 Step 1200: Loss = 0.07736613601446152 Accuracy = 0.9632077217102051
Epoch 2 Step 1400: Loss = 0.13602866232395172 Accuracy = 0.9640435576438904
Epoch 2 Step 1600: Loss = 0.1773453950881958 Accuracy = 0.9640263915061951
Epoch 2 Step 1800: Loss = 0.09336300939321518 Accuracy = 0.964793860912323


2024-12-14 19:33:58.525423: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


End of epoch 2, loss: 0.03442620113492012, accuracy: 0.9654499888420105


### Exercise 4: Add Hidden Layers 

Next, you will add a couple of hidden layers to your model. Hidden layers help the model learn complex patterns in the data. 


In [17]:
from tensorflow.keras.layers import Input, Dense

# Define the input layer
input_layer = Input(shape=(28, 28))  # Input layer with shape (28, 28)

# Define hidden layers
hidden_layer1 = Dense(64, activation='relu')(input_layer)  # First hidden layer with 64 neurons and ReLU activation
hidden_layer2 = Dense(64, activation='relu')(hidden_layer1)  # Second hidden layer with 64 neurons and ReLU activation


In the above code: 

`Dense(64, activation='relu')` creates a dense (fully connected) layer with 64 units and ReLU activation function. 

Each hidden layer takes the output of the previous layer as its input.


### Exercise 5: Define the output layer 

Finally, you will define the output layer. Suppose you are working on a binary classification problem, so the output layer will have one unit with a sigmoid activation function. 


In [18]:
output_layer = Dense(1, activation='sigmoid')(hidden_layer2)

In the above code: 

`Dense(1, activation='sigmoid')` creates a dense layer with 1 unit and a sigmoid activation function, suitable for binary classification. 


### Exercise 6: Create the Model 

Now, you will create the model by specifying the input and output layers. 


In [19]:
model = Model(inputs=input_layer, outputs=output_layer)

In the above code: 

`Model(inputs=input_layer, outputs=output_layer)` creates a Keras model that connects the input layer to the output layer through the hidden layers. 


### Exercise 7: Compile the Model 

Before training the model, you need to compile it. You will specify the loss function, optimizer, and evaluation metrics. 


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In the above code: 

`optimizer='adam'` specifies the Adam optimizer, a popular choice for training neural networks. 

`loss='binary_crossentropy'` specifies the loss function for binary classification problems. 

`metrics=['accuracy']` tells Keras to evaluate the model using accuracy during training. 


### Exercise 8: Train the Model 

You can now train the model on some training data. For this example, let's assume `X_train` is our training input data and `y_train` is the corresponding labels. 


In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
import numpy as np

# Step 1: Redefine the Model for 20 features
model = Sequential([
    Input(shape=(20,)),  # Adjust input shape to (20,)
    Dense(128, activation='relu'),  # Hidden layer with 128 neurons and ReLU activation
    Dense(1, activation='sigmoid')  # Output layer for binary classification with sigmoid activation
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Step 2: Generate Example Data
X_train = np.random.rand(1000, 20)  # 1000 samples, 20 features each
y_train = np.random.randint(2, size=(1000, 1))  # 1000 binary labels (0 or 1)

# Step 3: Train the Model
model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5282 - loss: 0.6967
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5241 - loss: 0.6936
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5481 - loss: 0.6813
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6090 - loss: 0.6796
Epoch 5/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5473 - loss: 0.6826
Epoch 6/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5654 - loss: 0.6764
Epoch 7/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5646 - loss: 0.6770
Epoch 8/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5809 - loss: 0.6712
Epoch 9/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6169 - loss: 0.6697
Epoch 10/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5587 - loss: 0.6785


In the above code: 

`X_train` and `y_train` are placeholders for your actual training data. 

`model.fit` trains the model for a specified number of epochs and batch size. 


### Exercise 9: Evaluate the Model 

After training, you can evaluate the model on test data to see how well it performs. 


In [21]:
# Example test data (in practice, use real dataset)
X_test = np.random.rand(200, 20)  # 200 samples, 20 features each
y_test = np.random.randint(2, size=(200, 1))  # 200 binary labels (0 or 1)

# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, y_test)

# Print test loss and accuracy
print(f'Test loss: {loss}')
print(f'Test accuracy: {accuracy}')


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5486 - loss: 0.6950  
Test loss: 0.7075135111808777
Test accuracy: 0.5199999809265137


In the above code: 

`model.evaluate` computes the loss and accuracy of the model on test data. 

`X_test` and `y_test` are placeholders for your actual test data. 


## Practice Exercises 

### Exercise 1: Basic Custom Training Loop 

#### Objective: Implement a basic custom training loop to train a simple neural network on the MNIST dataset. 

#### Instructions: 

- Set up the environment and load the dataset. 

- Define the model with a Flatten layer and two Dense layers. 

- Define the loss function and optimizer. 

- Implement a custom training loop to iterate over the dataset, compute the loss, and update the model's weights. 


In [ ]:

# Import necessary libraries
import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Flatten 

# Step 1: Set Up the Environment
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data() 
x_train, x_test = x_train / 255.0, x_test / 255.0 
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32) 

# Step 2: Define the Model
model = Sequential([ 
    Flatten(input_shape=(28, 28)), 
    Dense(128, activation='relu'), 
    Dense(10) 
]) 

# Step 3: Define Loss Function and Optimizer
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) 
optimizer = tf.keras.optimizers.Adam() 

# Step 4: Implement the Custom Training Loop
for epoch in range(5): 
    for x_batch, y_batch in train_dataset: 
        with tf.GradientTape() as tape: 
            logits = model(x_batch, training=True) 
            loss = loss_fn(y_batch, logits) 
        grads = tape.gradient(loss, model.trainable_weights) 
        optimizer.apply_gradients(zip(grads, model.trainable_weights)) 
    print(f'Epoch {epoch + 1}: Loss = {loss.numpy()}')

<details>
<summary>Click here for solution</summary> </br>

```python
# Import necessary libraries
import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Flatten 

# Step 1: Set Up the Environment
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data() 
x_train, x_test = x_train / 255.0, x_test / 255.0 
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32) 

# Step 2: Define the Model
model = Sequential([ 
    Flatten(input_shape=(28, 28)), 
    Dense(128, activation='relu'), 
    Dense(10) 
]) 

# Step 3: Define Loss Function and Optimizer
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) 
optimizer = tf.keras.optimizers.Adam() 

# Step 4: Implement the Custom Training Loop
for epoch in range(5): 
    for x_batch, y_batch in train_dataset: 
        with tf.GradientTape() as tape: 
            logits = model(x_batch, training=True) 
            loss = loss_fn(y_batch, logits) 
        grads = tape.gradient(loss, model.trainable_weights) 
        optimizer.apply_gradients(zip(grads, model.trainable_weights)) 
    print(f'Epoch {epoch + 1}: Loss = {loss.numpy()}')


### Exercise 2: Adding Accuracy Metric 

#### Objective: Enhance the custom training loop by adding an accuracy metric to monitor model performance. 

#### Instructions: 

1. Set up the environment and define the model, loss function, and optimizer. 

2. Add Sparse Categorical Accuracy as a metric. 

3. Implement the custom training loop with accuracy tracking.


In [ ]:
# Write your code here


<details>
<summary>Click here for solution</summary><br>

```python
# Import necessary libraries
import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Flatten 

# Step 1: Set Up the Environment
(x_train, y_train), _ = tf.keras.datasets.mnist.load_data() 
x_train = x_train / 255.0 
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32) 

# Step 2: Define the Model
model = Sequential([ 
    Flatten(input_shape=(28, 28)), 
    Dense(128, activation='relu'), 
    Dense(10) 
]) 

# Step 3: Define Loss Function, Optimizer, and Metric
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) 
optimizer = tf.keras.optimizers.Adam() 
accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy() 

# Step 4: Implement the Custom Training Loop with Accuracy Tracking
epochs = 5 
for epoch in range(epochs): 
    for x_batch, y_batch in train_dataset: 
        with tf.GradientTape() as tape: 
            logits = model(x_batch, training=True) 
            loss = loss_fn(y_batch, logits) 
        grads = tape.gradient(loss, model.trainable_weights) 
        optimizer.apply_gradients(zip(grads, model.trainable_weights)) 
        accuracy_metric.update_state(y_batch, logits) 
    print(f'Epoch {epoch + 1}: Loss = {loss.numpy()} Accuracy = {accuracy_metric.result().numpy()}') 
    accuracy_metric.reset_state() 


### Exercise 3: Custom Callback for Advanced Logging 

#### Objective: Implement a custom callback to log additional metrics and information during training. 

#### Instructions: 

1. Set up the environment and define the model, loss function, optimizer, and metric. 

2. Create a custom callback to log additional metrics at the end of each epoch. 

3. Implement the custom training loop with the custom callback. 


In [ ]:
# Write your code here


<details>
<summary>Click here for solution</summary> </br>

```python
# Import necessary libraries
import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Flatten 
from tensorflow.keras.callbacks import Callback 

# Step 1: Set Up the Environment
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data() 
x_train = x_train / 255.0 
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32) 

# Step 2: Define the Model
model = Sequential([ 
    tf.keras.Input(shape=(28, 28)),  # Updated Input layer syntax
    Flatten(), 
    Dense(128, activation='relu'), 
    Dense(10) 
]) 

# Step 3: Define Loss Function, Optimizer, and Metric
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) 
optimizer = tf.keras.optimizers.Adam() 
accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy() 

# Step 4: Implement the Custom Callback
class CustomCallback(Callback): 
    def on_epoch_end(self, epoch, logs=None): 
        print(f'End of epoch {epoch + 1}, loss: {logs.get("loss")}, accuracy: {logs.get("accuracy")}') 

# Step 5: Implement the Custom Training Loop with Custom Callback
custom_callback = CustomCallback() 

for epoch in range(5): 
    for x_batch, y_batch in train_dataset: 
        with tf.GradientTape() as tape: 
            logits = model(x_batch, training=True) 
            loss = loss_fn(y_batch, logits) 
        grads = tape.gradient(loss, model.trainable_weights) 
        optimizer.apply_gradients(zip(grads, model.trainable_weights)) 
        accuracy_metric.update_state(y_batch, logits) 
    custom_callback.on_epoch_end(epoch, logs={'loss': loss.numpy(), 'accuracy': accuracy_metric.result().numpy()}) 
    accuracy_metric.reset_state()  # Updated method



### Exercise 4: Lab - Hyperparameter Tuning 

#### Enhancement: Add functionality to save the results of each hyperparameter tuning iteration as JSON files in a specified directory. 

#### Additional Instructions:

Modify the tuning loop to save each iteration's results as JSON files.

Specify the directory where these JSON files will be stored for easier retrieval and analysis of tuning results.


In [ ]:
# Write your code here


<details>
<summary>Click here for solution</summary> </br>

```python
!pip install keras-tuner
!pip install scikit-learn

import json
import os
import keras_tuner as kt
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

# Step 1: Load your dataset
X, y = make_classification(n_samples=1000, n_features=20, n_classes=2)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

# Step 2: Define the model-building function
def build_model(hp):
    model = Sequential()
    # Tune the number of units in the first Dense layer
    model.add(Dense(units=hp.Int('units', min_value=32, max_value=512, step=32),
                    activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Binary classification example
    model.compile(optimizer=Adam(hp.Float('learning_rate', 1e-4, 1e-2, sampling='LOG')),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# Step 3: Initialize a Keras Tuner RandomSearch tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,  # Set the number of trials
    executions_per_trial=1,  # Set how many executions per trial
    directory='tuner_results',  # Directory for saving logs
    project_name='hyperparam_tuning'
)

# Step 4: Run the tuner search (make sure the data is correct)
tuner.search(X_train, y_train, validation_data=(X_val, y_val), epochs=5)

# Step 5: Save the tuning results as JSON files
try:
    for i in range(10):
        # Fetch the best hyperparameters from the tuner
        best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
        
        # Results dictionary to save hyperparameters and score
        results = {
            "trial": i + 1,
            "hyperparameters": best_hps.values,  # Hyperparameters tuned in this trial
            "score": None  # Add any score or metrics if available
        }

        # Save the results as JSON
        with open(os.path.join('tuning_results', f"trial_{i + 1}.json"), "w") as f:
            json.dump(results, f)

except IndexError:
    print("Tuning process has not completed or no results available.")
 ```   

</details>


### Exercise 5: Explanation of Hyperparameter Tuning

**Addition to Explanation:** Add a note explaining the purpose of num_trials in the hyperparameter tuning context:


In [ ]:
# Write your code here


<details>
<summary>Click here for solution</summary> </br>

```python
Explanation: "num_trials specifies the number of top hyperparameter sets to return. Setting num_trials=1 means that it will return only the best set of hyperparameters found during the tuning process."
 ```   

</details>


### Conclusion: 

Congratulations on completing this lab! You have now successfully created, trained, and evaluated a simple neural network model using the Keras Functional API. This foundational knowledge will allow you to build more complex models and explore advanced functionalities in Keras. 


Copyright © IBM Corporation. All rights reserved.
